**NOMBRE:** MARÍA ELIZABETH OLIVA GONZALES

# **Inicializar Spark**

In [ ]:
#INSTALACION

# instalar JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# instalar Spark
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

# instalar para usar con pypthon
!pip install -q findspark

la librería ***os*** proporciona una forma de interactuar con el sistema operativo. Proporciona una interfaz para trabajar con archivos, directorios, variables de entorno, procesos y otras funcionalidades relacionadas con el sistema operativo. Es parte de la biblioteca estándar de Python, lo que significa que no es necesario instalarla adicionalmente y está disponible en cualquier instalación de Python.

In [ ]:
# se establece estas variables de entorno antes de iniciar Spark en Python porque las usa para encontrar la ubicación de Java y Spark en el sistema operativo.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

**_findspark_** es una librería de Python que nos permite utilizar Spark en Python sin necesidad de instalarlo como una dependencia del sistema.

La ***función init()*** de findspark *establece las variables de entorno necesarias* para que Spark pueda ser utilizado desde Python, utilizando la ruta del directorio donde se encuentra Spark instalado en el sistema. Esto nos permite utilizar Spark en Python sin necesidad de configurar las variables de entorno manualmente cada vez que se inicia una sesión.

In [ ]:
import findspark
findspark.init()

## Parte 1

### 1. Crea un **DataFrame** a partir del siguiente conjunto de datos de ejemplo:



```
data = [("Alice", "F", 25),
  ("Bob", "M", 30),
  ("Charlie", "M", 45),
  ("Dave", "M", 50),
  ("Eve", "F", 35)]
df = spark.createDataFrame(data, ["nombre", "genero", "edad"])
df.show()

```



In [ ]:
# Importar libreria:
from pyspark.sql import SparkSession

# Inicializando sesion:
spark = SparkSession.builder.appName("nombres_df").getOrCreate()

In [ ]:
# definir los datos como una lista de tuplas:
data = [("Alice", "F", 25),
        ("Bob", "M", 30),
        ("Charlie", "M", 45),
        ("Dave", "M", 50),
        ("Eve", "F", 35)]

In [ ]:
# CREANDO EL DATA FRAME:
df = spark.createDataFrame(data, ["nombre", "genero", "edad"])
df.show() # mostrar df

+-------+------+----+
| nombre|genero|edad|
+-------+------+----+
|  Alice|     F|  25|
|    Bob|     M|  30|
|Charlie|     M|  45|
|   Dave|     M|  50|
|    Eve|     F|  35|
+-------+------+----+



### 2. Usa la función **printSchema()** para imprimir el esquema del DataFrame.

In [ ]:
df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- edad: long (nullable = true)



### 3. Usa la función **describe()** para obtener estadísticas descriptivas del DataFrame

**describe()** calcula automáticamente algunas estadísticas descriptivas del DataFrame, como:
- la cantidad de filas
- la media
- la desviación estándar
- el valor mínimo de cada columna numérica del DataFrame
- el valor máximo de cada columna numérica del DataFrame 
- las columnas no numéricas, solo se muestra la cantidad de filas.

In [ ]:
df.describe().show()

+-------+------+------+-----------------+
|summary|nombre|genero|             edad|
+-------+------+------+-----------------+
|  count|     5|     5|                5|
|   mean|  null|  null|             37.0|
| stddev|  null|  null|10.36822067666386|
|    min| Alice|     F|               25|
|    max|   Eve|     M|               50|
+-------+------+------+-----------------+



### 4. Usa la función **filter()** para seleccionar las filas donde el género es "M". Muestra las primeras 5 filas del DataFrame resultante.

**filter()**:
- No modifica el DataFrame original, sino que devuelve un nuevo DataFrame con las filas seleccionadas.
- Se puede usar en combinación con otras funciones de PySpark, como **select()**, **groupBy()**, **orderBy()**, **join()**, etc. para realizar operaciones más complejas en los DataFrames.

In [ ]:
df_m = df.filter(df.genero == "M")
df_m.show(5)

+-------+------+----+
| nombre|genero|edad|
+-------+------+----+
|    Bob|     M|  30|
|Charlie|     M|  45|
|   Dave|     M|  50|
+-------+------+----+



### 5. Usa la función **groupBy()** y la función **avg()** para calcular la edad promedio por género.

In [ ]:
# Importar la función avg() de PySpark para calcular la edad promedio
from pyspark.sql.functions import avg

La función **agg()** se utiliza para aplicar una o varias funciones de agregación (**como avg(), count(), sum(),** etc.) a los datos de un grupo específico. 

In [ ]:
# se agrupa por la columna "genero" y se aplica la función avg() a la columna "edad".
df_avg = df.groupBy("genero").agg(avg("edad"))
# mostrar el dataframe
df_avg.show()

+------+------------------+
|genero|         avg(edad)|
+------+------------------+
|     F|              30.0|
|     M|41.666666666666664|
+------+------------------+



### 6. Usa la función **withColumn()** para crear una nueva columna llamada **grupo_edad** que mapee la edad de cada persona a un grupo de edad. Por ejemplo, si la edad de una persona es 25, su grupo de edad sería "20-29". *Muestra las primeras 5 filas del DataFrame resultante.*

In [ ]:
from pyspark.sql.functions import when

# Definir las condiciones de cada grupo de edad
condiciones = [ 
    (df.edad >= 20) & (df.edad <= 29),
    (df.edad >= 30) & (df.edad <= 39),
    (df.edad >= 40) & (df.edad <= 49),
    (df.edad >= 50)
]

# Definir las etiquetas para cada grupo de edad
etiquetas = ["20-29", "30-39", "40-49", "50+"]

# Usar la función when() para mapear cada edad a un grupo de edad
df_grupo_edad = df.withColumn("grupo_edad", 
                              when(condiciones[0], etiquetas[0])
                              .when(condiciones[1], etiquetas[1])
                              .when(condiciones[2], etiquetas[2])
                              .when(condiciones[3], etiquetas[3]))

# Mostrar las primeras 5 filas del DataFrame resultante
df_grupo_edad.show(5)


+-------+------+----+----------+
| nombre|genero|edad|grupo_edad|
+-------+------+----+----------+
|  Alice|     F|  25|     20-29|
|    Bob|     M|  30|     30-39|
|Charlie|     M|  45|     40-49|
|   Dave|     M|  50|       50+|
|    Eve|     F|  35|     30-39|
+-------+------+----+----------+



### 7. Usa la función **agg()** para calcular la cantidad de personas en cada grupo de edad.

**agg()** 
- es una herramienta esencial para calcular estadísticas y realizar operaciones de agregación en PySpark.
- se utiliza comúnmente junto con la función groupBy(), que se utiliza para agrupar los datos en el DataFrame según una o varias columnas.

In [ ]:
from pyspark.sql.functions import count

# Agrupar el DataFrame por la columna "grupo_edad" y contar la cantidad de registros en cada grupo
df_grupo_edad_count = df_grupo_edad.groupBy("grupo_edad").agg(count("*").alias("cantidad_personas"))

# Mostrar el DataFrame resultante
df_grupo_edad_count.show()


+----------+-----------------+
|grupo_edad|cantidad_personas|
+----------+-----------------+
|     30-39|                2|
|     20-29|                1|
|       50+|                1|
|     40-49|                1|
+----------+-----------------+



## Parte 2

### 1. Crea el siguiente **dataframe:**
```
data = [("Bob", 25, "M", 50000),
  ("Alice", 32, "F", 60000),
  ("Charlie", 45, "M", 80000),
  ("Dave", 50, "M", 75000),
  ("Eve", 35, "F", 70000)]

df = spark.createDataFrame(data, ["nombre", "edad", "genero", "salario"])
df.show()
```

In [ ]:
from pyspark.sql import SparkSession
# Crear una sesión de Spark
spark = SparkSession.builder.appName("personas").getOrCreate()

# Crear los datos para el DataFrame
data = [("Bob", 25, "M", 50000), 
 ("Alice", 32, "F", 60000), 
 ("Charlie", 45, "M", 80000), 
 ("Dave", 50, "M", 75000), 
 ("Eve", 35, "F", 70000)]

# Crear el DataFrame a partir de los datos
df = spark.createDataFrame(data, ["nombre", "edad", "genero", "salario"])
# Mostrar el DataFrame
df.show()

+-------+----+------+-------+
| nombre|edad|genero|salario|
+-------+----+------+-------+
|    Bob|  25|     M|  50000|
|  Alice|  32|     F|  60000|
|Charlie|  45|     M|  80000|
|   Dave|  50|     M|  75000|
|    Eve|  35|     F|  70000|
+-------+----+------+-------+



### 2. Usa la función **select()** para seleccionar las columnas nombre y salario. Muestra las primeras 5 filas del DataFrame resultante.

In [ ]:
# Seleccionar las columnas nombre y salario
df_nombres_salarios = df.select("nombre", "salario")
# Mostrar las primeras 5 filas del DataFrame resultante
df_nombres_salarios.show(5)

+-------+-------+
| nombre|salario|
+-------+-------+
|    Bob|  50000|
|  Alice|  60000|
|Charlie|  80000|
|   Dave|  75000|
|    Eve|  70000|
+-------+-------+



### 3. Usa la función **orderBy()** para ordenar el DataFrame por edad de forma descendente. Muestra las primeras 5 filas del DataFrame resultante.

In [ ]:
# Ordenar el DataFrame por edad de forma descendente
df_edad_descendente = df.orderBy(df.edad.desc())

# Mostrar las primeras 5 filas del DataFrame resultante
df_edad_descendente.show(5)

+-------+----+------+-------+
| nombre|edad|genero|salario|
+-------+----+------+-------+
|   Dave|  50|     M|  75000|
|Charlie|  45|     M|  80000|
|    Eve|  35|     F|  70000|
|  Alice|  32|     F|  60000|
|    Bob|  25|     M|  50000|
+-------+----+------+-------+



### 4. Usa la función **filter()** para seleccionar las filas donde el género es "F" y el salario es mayor que 65000. Muestra las primeras 5 filas del DataFrame resultante.

In [ ]:
# Seleccionar las filas donde el género es "F" y el salario es mayor que 65000
df_f_salario = df.filter((df.genero == "F") & (df.salario > 65000))

# Mostrar las primeras 5 filas del DataFrame resultante
df_f_salario.show(5)

+------+----+------+-------+
|nombre|edad|genero|salario|
+------+----+------+-------+
|   Eve|  35|     F|  70000|
+------+----+------+-------+



### 5. Usa la función **groupBy()** y la función **sum()** para calcular la suma total de salarios por género. 

In [ ]:
from pyspark.sql.functions import sum
# Calcular la suma total de salarios por género
df_suma_salarios = df.groupBy("genero").sum("salario")

# Mostrar el DataFrame resultante
df_suma_salarios.show()

+------+------------+
|genero|sum(salario)|
+------+------------+
|     F|      130000|
|     M|      205000|
+------+------------+



### 6. Usa la función **withColumn()** para crear una nueva columna llamada ***salario_anual*** que muestre el salario anual de cada persona. Muestra las primeras 5 filas del DataFrame resultante.

In [ ]:
# Crear una nueva columna llamada "salario_anual" que muestre el salario anual de cada persona
df_con_salario_anual = df.withColumn("salario_anual", df.salario * 12)

# Mostrar las primeras 5 filas del DataFrame resultante
df_con_salario_anual.show(5)

+-------+----+------+-------+-------------+
| nombre|edad|genero|salario|salario_anual|
+-------+----+------+-------+-------------+
|    Bob|  25|     M|  50000|       600000|
|  Alice|  32|     F|  60000|       720000|
|Charlie|  45|     M|  80000|       960000|
|   Dave|  50|     M|  75000|       900000|
|    Eve|  35|     F|  70000|       840000|
+-------+----+------+-------+-------------+



### 7. Usa la función **agg()** para calcular la edad promedio y el salario máximo del DataFrame.

In [ ]:
from pyspark.sql.functions import avg, max
# Mostrar el DataFrame inicial
df.show()

+-------+----+------+-------+
| nombre|edad|genero|salario|
+-------+----+------+-------+
|    Bob|  25|     M|  50000|
|  Alice|  32|     F|  60000|
|Charlie|  45|     M|  80000|
|   Dave|  50|     M|  75000|
|    Eve|  35|     F|  70000|
+-------+----+------+-------+



In [ ]:
# Calcular la edad promedio y el salario máximo del DataFrame
resultados = df.agg({"edad": "avg", "salario": "max"})
# Mostrar el DataFrame resultante
resultados.show()

+------------+---------+
|max(salario)|avg(edad)|
+------------+---------+
|       80000|     37.4|
+------------+---------+



### 8. Usa la función **groupBy()** y la **función count()** para contar el número de personas por género.

In [ ]:
from pyspark.sql.functions import count
# Contar el número de personas por género
conteo_por_genero = df.groupBy("genero").count()
# Mostrar el DataFrame resultante
conteo_por_genero.show()

+------+-----+
|genero|count|
+------+-----+
|     F|    2|
|     M|    3|
+------+-----+



### 9. Usa la función **filter()** para seleccionar las filas donde la edad está entre 30 y 40 años (inclusive). Muestra las primeras 5 filas del DataFrame resultante

In [ ]:
# Seleccionar las filas donde la edad está entre 30 y 40 años
filtro_edad = df.filter((df.edad >= 30) & (df.edad <= 40))
# Mostrar las primeras 5 filas del DataFrame resultante
filtro_edad.show(5)

+------+----+------+-------+
|nombre|edad|genero|salario|
+------+----+------+-------+
| Alice|  32|     F|  60000|
|   Eve|  35|     F|  70000|
+------+----+------+-------+



### 10. Usa la función **withColumnRenamed()** para cambiar el nombre de la columna "edad" a "años". Muestra las primeras 5 filas del DataFrame resultante.

In [ ]:
# Mostrar el DataFrame inicial
df.show()

+-------+----+------+-------+
| nombre|edad|genero|salario|
+-------+----+------+-------+
|    Bob|  25|     M|  50000|
|  Alice|  32|     F|  60000|
|Charlie|  45|     M|  80000|
|   Dave|  50|     M|  75000|
|    Eve|  35|     F|  70000|
+-------+----+------+-------+



In [ ]:
# Renombrar la columna "edad" a "años"
df_con_nombres_nuevos = df.withColumnRenamed("edad", "años")

# Mostrar las primeras 5 filas del DataFrame resultante
df_con_nombres_nuevos.show(5)

+-------+----+------+-------+
| nombre|años|genero|salario|
+-------+----+------+-------+
|    Bob|  25|     M|  50000|
|  Alice|  32|     F|  60000|
|Charlie|  45|     M|  80000|
|   Dave|  50|     M|  75000|
|    Eve|  35|     F|  70000|
+-------+----+------+-------+



### 11. Usa la función when() para crear una nueva columna llamada  "categoria_salarial" que tenga los valores "bajo" para salarios menores o iguales a 60000, "medio" para salarios mayores que 60000 y menores o iguales a 75000, y "alto" para salarios mayores que 75000. Muestra las primeras 5 filas del DataFrame resultante.

In [ ]:
from pyspark.sql.functions import when

# Crear la nueva columna "categoria_salarial" utilizando la función when()
df_con_categorias = df.withColumn("categoria_salarial",
                                  when(df["salario"] <= 60000, "bajo")
                                  .when((df["salario"] > 60000) & (df["salario"] <= 75000), "medio")
                                  .otherwise("alto"))

# Mostrar las primeras 5 filas del DataFrame resultante
df_con_categorias.show(5)

+-------+----+------+-------+------------------+
| nombre|edad|genero|salario|categoria_salarial|
+-------+----+------+-------+------------------+
|    Bob|  25|     M|  50000|              bajo|
|  Alice|  32|     F|  60000|              bajo|
|Charlie|  45|     M|  80000|              alto|
|   Dave|  50|     M|  75000|             medio|
|    Eve|  35|     F|  70000|             medio|
+-------+----+------+-------+------------------+



### 12. Usa la función approxQuantile() para calcular el percentil 25 y el percentil 75 del salario.

In [ ]:
#Calcular los percentiles 25 y 75 del salario
percentiles = df.approxQuantile("salario", [0.25, 0.75], 0)

#Mostrar los percentiles calculados
print("Percentil 25 del salario:", percentiles[0])
print("Percentil 75 del salario:", percentiles[1])

Percentil 25 del salario: 60000.0
Percentil 75 del salario: 75000.0


## **Extra:**

### 1. Supongamos que tenemos un conjunto de datos con información sobre películas que incluye los siguientes campos:

* **title:** el título de la película
* **year:** el año de estreno de la película
* **genre:** el género de la película
* **runtime:** la duración en minutos de la película
* **rating:** la calificación promedio de la película en una escala del 1 al 10
* **votes:** el número de votos que recibió la película

In [ ]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

# Crear los datos para el DataFrame
data = [("The Shawshank Redemption", 1994, "Drama", 142, 9.3, 9998),
        ("The Godfather", 1972, "Crime", 175, 9.2, 8888),
        ("The Dark Knight", 2008, "Drama", 152, 9.0, 7777),
        ("The Lord of the Rings: The Return of the King", 2003, "Action", 201, 8.9, 6666),
        ("Pulp Fiction", 1994, "Crime", 154, 8.9, 5555),
        ("Schindler's List", 1993, "Biography", 195, 8.9, 4444),
        ("The Lord of the Rings: The Fellowship of the Ring", 2002, "Action", 179, 8.8, 3333),
        ("Forrest Gump", 1994, "Comedy", 142, 8.8, 2222),
        ("Inception", 2010, "Action", 148, 8.8, 1111)]
        
# Crear el DataFrame a partir de los datos
df = spark.createDataFrame(data, ["title", "year", "genre", "runtime", "rating"])

# Mostrar el DataFrame
df.show()

+--------------------+----+---------+-------+------+----+
|               title|year|    genre|runtime|rating|  _6|
+--------------------+----+---------+-------+------+----+
|The Shawshank Red...|1994|    Drama|    142|   9.3|9998|
|       The Godfather|1972|    Crime|    175|   9.2|8888|
|     The Dark Knight|2008|    Drama|    152|   9.0|7777|
|The Lord of the R...|2003|   Action|    201|   8.9|6666|
|        Pulp Fiction|1994|    Crime|    154|   8.9|5555|
|    Schindler's List|1993|Biography|    195|   8.9|4444|
|The Lord of the R...|2002|   Action|    179|   8.8|3333|
|        Forrest Gump|1994|   Comedy|    142|   8.8|2222|
|           Inception|2010|   Action|    148|   8.8|1111|
+--------------------+----+---------+-------+------+----+



### 2. Usa la función concat para crear una nueva columna llamada **title_year** que contenga el título de la película seguido del año de estreno, separados por un espacio.

In [ ]:
from pyspark.sql.functions import concat

# Agregar la columna title_year
df = df.withColumn("title_year", concat("title", "year"))

# Mostrar el DataFrame actualizado
df.show()

+--------------------+----+---------+-------+------+----+--------------------+
|               title|year|    genre|runtime|rating|  _6|          title_year|
+--------------------+----+---------+-------+------+----+--------------------+
|The Shawshank Red...|1994|    Drama|    142|   9.3|9998|The Shawshank Red...|
|       The Godfather|1972|    Crime|    175|   9.2|8888|   The Godfather1972|
|     The Dark Knight|2008|    Drama|    152|   9.0|7777| The Dark Knight2008|
|The Lord of the R...|2003|   Action|    201|   8.9|6666|The Lord of the R...|
|        Pulp Fiction|1994|    Crime|    154|   8.9|5555|    Pulp Fiction1994|
|    Schindler's List|1993|Biography|    195|   8.9|4444|Schindler's List1993|
|The Lord of the R...|2002|   Action|    179|   8.8|3333|The Lord of the R...|
|        Forrest Gump|1994|   Comedy|    142|   8.8|2222|    Forrest Gump1994|
|           Inception|2010|   Action|    148|   8.8|1111|       Inception2010|
+--------------------+----+---------+-------+------+

### 3.Usa la función substring para crear una nueva columna llamada first_letter que contenga la primera letra del género de la película. El resultado debe ser el siguiente:

In [ ]:
from pyspark.sql.functions import substring

# Agregar la columna first_letter
df = df.withColumn("first_letter", substring("genre", 1, 1))
#Función substring extrae la primera letra de la columna "genre" y agregarla a una nueva columna llamada "first_letter".

# Mostrar el DataFrame actualizado
df.show()

+--------------------+----+---------+-------+------+----+--------------------+------------+
|               title|year|    genre|runtime|rating|  _6|          title_year|first_letter|
+--------------------+----+---------+-------+------+----+--------------------+------------+
|The Shawshank Red...|1994|    Drama|    142|   9.3|9998|The Shawshank Red...|           D|
|       The Godfather|1972|    Crime|    175|   9.2|8888|   The Godfather1972|           C|
|     The Dark Knight|2008|    Drama|    152|   9.0|7777| The Dark Knight2008|           D|
|The Lord of the R...|2003|   Action|    201|   8.9|6666|The Lord of the R...|           A|
|        Pulp Fiction|1994|    Crime|    154|   8.9|5555|    Pulp Fiction1994|           C|
|    Schindler's List|1993|Biography|    195|   8.9|4444|Schindler's List1993|           B|
|The Lord of the R...|2002|   Action|    179|   8.8|3333|The Lord of the R...|           A|
|        Forrest Gump|1994|   Comedy|    142|   8.8|2222|    Forrest Gump1994|  

### 4.Usa la función 'when' para crear una nueva columna llamada 'highly_rated' que tenga el valor 'True' si la calificación de la película es mayor a 9, y 'False' en caso contrario. El resultado debe ser el siguiente:

In [ ]:
# Agregar la columna highly_rated
df = df.withColumn("highly_rated", when(df["rating"] > 9, True).otherwise(False))
# Mostrar el DataFrame actualizado
df.show()

+--------------------+----+---------+-------+------+----+--------------------+------------+------------+
|               title|year|    genre|runtime|rating|  _6|          title_year|first_letter|highly_rated|
+--------------------+----+---------+-------+------+----+--------------------+------------+------------+
|The Shawshank Red...|1994|    Drama|    142|   9.3|9998|The Shawshank Red...|           D|        true|
|       The Godfather|1972|    Crime|    175|   9.2|8888|   The Godfather1972|           C|        true|
|     The Dark Knight|2008|    Drama|    152|   9.0|7777| The Dark Knight2008|           D|       false|
|The Lord of the R...|2003|   Action|    201|   8.9|6666|The Lord of the R...|           A|       false|
|        Pulp Fiction|1994|    Crime|    154|   8.9|5555|    Pulp Fiction1994|           C|       false|
|    Schindler's List|1993|Biography|    195|   8.9|4444|Schindler's List1993|           B|       false|
|The Lord of the R...|2002|   Action|    179|   8.8|333

### 5. Usa la función filter para seleccionar solamente las películas que duran más de 150 minutos.

In [ ]:
from pyspark.sql.functions import col

# Aplicar la función filter para seleccionar películas con duración mayor a 150 minutos
df_filtered = df.filter(col("runtime") > 150)

# Mostrar el DataFrame resultante
df_filtered.show()

+--------------------+----+---------+-------+------+----+--------------------+------------+------------+
|               title|year|    genre|runtime|rating|  _6|          title_year|first_letter|highly_rated|
+--------------------+----+---------+-------+------+----+--------------------+------------+------------+
|       The Godfather|1972|    Crime|    175|   9.2|8888|   The Godfather1972|           C|        true|
|     The Dark Knight|2008|    Drama|    152|   9.0|7777| The Dark Knight2008|           D|       false|
|The Lord of the R...|2003|   Action|    201|   8.9|6666|The Lord of the R...|           A|       false|
|        Pulp Fiction|1994|    Crime|    154|   8.9|5555|    Pulp Fiction1994|           C|       false|
|    Schindler's List|1993|Biography|    195|   8.9|4444|Schindler's List1993|           B|       false|
|The Lord of the R...|2002|   Action|    179|   8.8|3333|The Lord of the R...|           A|       false|
+--------------------+----+---------+-------+------+---

### 6. Usa la función **groupBy** y la función avg para calcular el promedio de calificación de las películas por género.

In [ ]:
from pyspark.sql.functions import avg

# Calcular el promedio de calificación por género
avg_ratings_by_genre = df.groupBy("genre").agg(avg("rating").alias("avg_rating"))

# Mostrar el DataFrame resultante
avg_ratings_by_genre.show()

+---------+-----------------+
|    genre|       avg_rating|
+---------+-----------------+
|    Crime|             9.05|
|    Drama|             9.15|
|Biography|              8.9|
|   Comedy|              8.8|
|   Action|8.833333333333334|
+---------+-----------------+

